<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/ANMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplos de ANMI

In [1]:
from sympy import *
from sympy.matrices import Matrix as mat
import numpy as np
from numpy.linalg import cond as numero_condicion
from sympy.matrices import randMatrix

In [2]:
M = mat(((1, 1, 1), (2, -1, 1), (1, 2, -2)))
M = mat(((1, 2, 3, 4), (5, 6, 7, 8), (9, 10, 11, 11), (132, 14, 15, 170)))
# M = mat(((2, 3 , 1), (1, 1 , 4), (4, 4, 1)))


In [3]:
mat(((1, 0 , 0), (1/2, 1 , 0), (1/4, 0, 1))) * mat(((4, 4, 1), (0, 1, 1/2), (0, 0, 15/4)))

Matrix([
[  4,   4,   1],
[2.0, 3.0, 1.0],
[1.0, 1.0, 4.0]])

## Tema 2

In [71]:
def permutacion_matriz(U, fila_i, idx_max, verbose, P=None, r=None):
      if verbose:
        print(f'Permutamos fila {fila_i} con {idx_max[0]}')
        print(f'U antes:\n {np.array(U)}')
        print(f'P antes:\n {np.array(P)}')

      if fila_i != idx_max[0]:
        fila_origen, fila_destino = U[fila_i, :].copy(), U[idx_max[0], :].copy()
        U[idx_max[0], :], U[fila_i, :] = fila_origen, fila_destino
        if P is not None:
          fila_origen, fila_destino = P[fila_i, :].copy(), P[idx_max[0], :].copy()
          P[idx_max[0], :], P[fila_i, :] = fila_origen, fila_destino
        if r is not None:
          fila_origen, fila_destino = r[fila_i, :].copy(), r[idx_max[0], :].copy()
          r[idx_max[0], :], r[fila_i, :] = fila_origen, fila_destino

      if verbose:
         print(f'U despues:\n {np.array(U)}')
         print(f'P despues:\n {np.array(P)}')
      return U, P, r

def permutacion_L(L, lista_perms, verbose):
  if verbose:
    print(f'L antes:\n {np.array(L)}')

  for perm in lista_perms:
    fila_origen, fila_destino = L[perm[0], :min(perm)].copy(), L[perm[1], :min(perm)].copy()
    L[perm[1], :min(perm)], L[perm[0], :min(perm)] = fila_origen, fila_destino

  if verbose:
    print(f'L despues:\n {np.array(L)}')
  return L 





def descomposicion_LU(m, rhs=None, verbose=True, permutar_max=False):
  '''
  Esta función realiza el algoritmo de triangulación de Gauss. Para ello vamos a
  ir aplicando paso a paso el algoritmo tal cual se hace manualmente, 
  y aplicamos los cambios de columnas necesarios si hay que aplicar permutaciones.
  Por defecto, si encontramos un 0 en la diagonal aplicamos permutar_max para esa
  fila, y devolvemos la matriz de permutaciones.
  '''

  if rhs is None:
    rhs = zeros(m.shape[0], 1)

  if verbose:
    print("La matriz M|X es  (X = 0) si no se ha introducido")
    print(np.concatenate((np.array(m), np.array(rhs)), axis=1))
  
  P, L, U, r = eye(m.shape[0]), eye(m.shape[0]), m.copy(), rhs.copy()
  lista_perms = []
  fila_i, err, err_max = 0, 0, 3
  

  while fila_i < m.shape[0] and err < err_max:
    if verbose:
      print(f'\n=====================================\nFila {fila_i}')
      print(f'A {fila_i}, {err}')

    if U[max(0, fila_i-1), max(0, fila_i-1)] == 0 or permutar_max:
      """
      Esta parte nos asegura que si un elemento diagonal es cero, permutamos
      la fila con su inmediata, y así tener una configuración viable.
      En cualquier permutación tenemos que cambiar la L acorde con el cambio.
      Al hacer la permutación, con los nuevos valores, repasamos la matriz para 
      asegurarnos de que todos los puntos están cumplidos.
      """
      sub_mat = U[fila_i:, fila_i:]
      max_el = np.max(np.array(sub_mat))
      idx_max = np.argwhere(np.array(sub_mat) == max_el)[-1] + fila_i # Cogemos el primer elemento si hay varios
      
      U, P, r = permutacion_matriz(U, max(0, fila_i-1), idx_max, verbose, P, r)
      L = permutacion_L(L, [[max(0, fila_i-1), idx_max[0]]], verbose)
      lista_perms.append([max(0, fila_i-1), idx_max[0]])
      fila_i = min([max(0, fila_i-1), idx_max[0]])
      err += 1

      if verbose:
        print(f'Err {err}')
      continue
    
    # Ahora aplicamos el algoritmo de calculo de filas:
    for columna_j in range(fila_i):
      a_ij = U[fila_i, columna_j] /  U[columna_j, columna_j]
      
      if a_ij != 0:
        L[fila_i, columna_j] = a_ij
        U[fila_i, :] = U[fila_i, :] - a_ij * U[columna_j, :]
        r[fila_i, :] = r[fila_i, :] - a_ij * r[columna_j, :]

      err = 0

      if verbose:
        print(f'||||||||||||||||||||||||\n Columna {columna_j}')
        print(f'a_{fila_i},{columna_j} = {a_ij}')
        print(f'P = \n{np.array(P)}')
        print(f'L = \n{np.array(L)}')
        print(f'U = \n{np.array(U)}')
        print(f'r = \n{np.array(r)}')

    fila_i += 1
  
  if err == err_max:
    print('Something has gone wrong... check the log.')

  if verbose:
        print(f'\/\/\/\/\/\/\/\/\/ FORMA FINAL')
        print(f'P = \n{np.array(P)}')
        print(f'L = \n{np.array(L)}')
        print(f'U = \n{np.array(U)}')
        print(f'r = \n{np.array(r)}')

  assert L * U == P * m

  return {'P': P, 'L': L, 'U': U, 'r': r}




def descomposicion_LDU(m, permutar_max=True, verbose=False):
  dict_LU = descomposicion_LU(m, permutar_max=permutar_max, verbose=verbose)
  L, U = dict_LU['L'], dict_LU['U']

  D = zeros(m.shape[0], m.shape[0])

  for i in range(U.shape[0]):
    D[i, i] = U[i, i]
  
  U = D.inv() * U

  if verbose:
        print(f'\/\/\/\/\/\/\/\/\/ RESULTS LDU*')
        print(f'L = \n{np.array(L)}')
        print(f'D = \n{np.array(D)}')
        print(f'U* = \n{np.array(U)}')

  assert L * D * U == dict_LU['P'] * m

  return {'P': dict_LU['P'], 'L': L, 'U': U, 'D': D}
  
  


In [4]:
numero_condicion(np.array(M, dtype=float))

3114.6258933625754

In [72]:
descomposicion_LU(M, permutar_max=False)

La matriz M|X es  (X = 0) si no se ha introducido
[[1 2 3 4 0]
 [5 6 7 8 0]
 [9 10 11 11 0]
 [132 14 15 170 0]]

Fila 0
A 0, 0

Fila 1
A 1, 0
||||||||||||||||||||||||
 Columna 0
a_1,0 = 5
P = 
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
L = 
[[1 0 0 0]
 [5 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
U = 
[[1 2 3 4]
 [0 -4 -8 -12]
 [9 10 11 11]
 [132 14 15 170]]
r = 
[[0]
 [0]
 [0]
 [0]]

Fila 2
A 2, 0
||||||||||||||||||||||||
 Columna 0
a_2,0 = 9
P = 
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
L = 
[[1 0 0 0]
 [5 1 0 0]
 [9 0 1 0]
 [0 0 0 1]]
U = 
[[1 2 3 4]
 [0 -4 -8 -12]
 [0 -8 -16 -25]
 [132 14 15 170]]
r = 
[[0]
 [0]
 [0]
 [0]]
||||||||||||||||||||||||
 Columna 1
a_2,1 = 2
P = 
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
L = 
[[1 0 0 0]
 [5 1 0 0]
 [9 2 1 0]
 [0 0 0 1]]
U = 
[[1 2 3 4]
 [0 -4 -8 -12]
 [0 0 0 -1]
 [132 14 15 170]]
r = 
[[0]
 [0]
 [0]
 [0]]

Fila 3
A 3, 0
Permutamos fila 2 con 3
U antes:
 [[1 2 3 4]
 [0 -4 -8 -12]
 [0 0 0 -1]
 [132 14 15 170]]
P antes:
 [[1 0 0 0]
 [0 1 0 0]
 

{'L': Matrix([
 [  1,     0, 0, 0],
 [  5,     1, 0, 0],
 [132, 125/2, 1, 0],
 [  9,     2, 0, 1]]), 'P': Matrix([
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0]]), 'U': Matrix([
 [1,  2,   3,   4],
 [0, -4,  -8, -12],
 [0,  0, 119, 392],
 [0,  0,   0,  -1]]), 'r': Matrix([
 [0],
 [0],
 [0],
 [0]])}

In [43]:
descomposicion_LDU(M, permutar_max=True)

Something has gone wrong... check the log.
[[132 14 15 170]
 [5 6 7 8]
 [9 10 11 11]
 [1 2 3 4]] // [[132 14 15 170]
 [5 6 7 8]
 [9 10 11 11]
 [1 2 3 4]] // [[1 2 3 4]
 [5 6 7 8]
 [9 10 11 11]
 [132 14 15 170]]


TypeError: ignored

In [ ]:
descomposicion_LU(M, rhs=ones(M.shape[0], 1), permutar_max=True)

In [31]:
descomposicion_LDU(M, permutar_max=True)

La matriz M|X es  (X = 0) si no se ha introducido


ValueError: ignored

In [ ]:
a = np.array([[1, 2, 3, 4], [2, 1, 2, 3], [3, 2, 1, 2], [4, 3, 2, 1]])

In [ ]:
eig = np.linalg.eigvals(a)

print(eig)

[ 9.09901951 -3.41421356 -1.09901951 -0.58578644]


In [ ]:
np.all(np.linalg.eigvals(a) > 0)

False

In [ ]:
from sympy.matrices import Matrix

In [ ]:
A = Matrix(((25, 15, -5), (15, 18, 0), (-5, 0, 10)))

In [ ]:
A_chol = A.cholesky()

In [ ]:
A.det()

1800

In [ ]:
A_chol * A_chol.T

Matrix([
[25, 15, -5],
[15, 18,  0],
[-5,  0, 10]])

In [ ]:
B = Matrix.eye(10)

In [ ]:
for i in range(9):
  B[i, i+1] = 2
  B[i+1, i] = 2

In [ ]:
B

Matrix([
[1, 2, 0, 0, 0, 0, 0, 0, 0, 0],
[2, 1, 2, 0, 0, 0, 0, 0, 0, 0],
[0, 2, 1, 2, 0, 0, 0, 0, 0, 0],
[0, 0, 2, 1, 2, 0, 0, 0, 0, 0],
[0, 0, 0, 2, 1, 2, 0, 0, 0, 0],
[0, 0, 0, 0, 2, 1, 2, 0, 0, 0],
[0, 0, 0, 0, 0, 2, 1, 2, 0, 0],
[0, 0, 0, 0, 0, 0, 2, 1, 2, 0],
[0, 0, 0, 0, 0, 0, 0, 2, 1, 2],
[0, 0, 0, 0, 0, 0, 0, 0, 2, 1]])

In [ ]:
A.eigenvals()

{53/3 + (-1/2 - sqrt(3)*I/2)*(22922/27 + 5*sqrt(123819)*I/3)**(1/3) + 919/(9*(-1/2 - sqrt(3)*I/2)*(22922/27 + 5*sqrt(123819)*I/3)**(1/3)): 1,
 53/3 + 919/(9*(-1/2 + sqrt(3)*I/2)*(22922/27 + 5*sqrt(123819)*I/3)**(1/3)) + (-1/2 + sqrt(3)*I/2)*(22922/27 + 5*sqrt(123819)*I/3)**(1/3): 1,
 53/3 + 919/(9*(22922/27 + 5*sqrt(123819)*I/3)**(1/3)) + (22922/27 + 5*sqrt(123819)*I/3)**(1/3): 1}